## EDA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import pandas as pd
import gc #garbage collector

In [ ]:
# Importa los datasets de score y titles
score = pd.read_csv('ignore/score.csv')
titles = pd.read_csv('data/all.csv')

In [ ]:
print(score.shape)
score.tail(2)

In [ ]:
titles.tail(2) # No coinciden los encabezados de ambos datasets para movieId e id

In [ ]:

# Filtra las columnas userId, movieId y score del dataset score y las columnas id y title del dataset title.
score = score[['userId','movieId','score','platform']]
titles = titles[['id','title']]
titles = titles.rename(columns={'id':'movieId'}) # Cambia el nombre para que haya coincidencia
df_title = titles[['movieId','title']]
df_title.tail(2)

In [ ]:
# Unifica los datasets
df = pd.merge(score,titles,how='left',on='movieId')
scores_amount = len(df)-1
df.tail(2)

In [ ]:
# Score por plataforma y por pelicula (min = 3.3, max = 3.72)
scoreByP = df.groupby(by=['movieId','platform']).mean()
maxAvgScore = scoreByP['score'].max()
minAvgScore = scoreByP['score'].min()
scoreByP.head(2)

In [ ]:
id_max = scoreByP['score'].idxmax()[0]
movie = df.iloc[id_max].title

# idx_max = df['userId'].idxmax() # 9524029
# movieMaxAmountScores = df.loc[idx_max].title

# scoreByP['score'].idxmax()[0]

In [ ]:
# Usuarios únicos 115077
print('users:',len(score['userId'].unique()))
usersAmount = len(score['userId'].unique())
# Calificaciones de películas por usuario
scoresByUser = score.groupby(['userId']).count()
scoresByUser.tail()

In [ ]:
# Distribucion de calificaciones por usuario
plt.hist(scoresByUser.score, log= True)
plt.title('Scores distribution by userId')  
plt.ylabel("Scores Amount")      # Adjust the label of the y-axis
plt.xlabel("Users Id")              # # Adjust the label of the x-axis
plt.show()

In [ ]:
# Distribución de las cantidades de calificaciones
score['score'].hist()
plt.title('Scores distribution by score')
plt.ylabel("Scores Amount")      # Adjust the label of the y-axis
plt.xlabel("Score")              # # Adjust the label of the x-axis
plt.show()

In [ ]:
# Calificaciones por pelicula
print('movies:',len(score['movieId'].unique()))     # Cantidad de peliculas/series: 22998
scoresByMovie = score.groupby(['movieId']).count()  # Agrupar por peliculas/series  
maxAmountScores = scoresByMovie['score'].max()      # Agrupacion maxima: 576
scoresByMovie.head(2)

In [ ]:
# Peliculas/series con mas y menos calificaciones
idx_max = scoresByMovie['userId'].idxmax()                  # Index de maxima cantidad de scores (576): as4197
print('movieId with maximum amount of scores:',idx_max) 
maxAmountScores = scoresByMovie['score'].max()              # Maxima cantidad de scores: 576
movieIdMaxAmountScores = idx_max                            # movieId con maxima cantidad de scores: as4197
idx_min = scoresByMovie['userId'].idxmin()                  # Index de minimo Id de usuario (396): as4940
print('movieId with minimum amount of scores:',idx_min)
minAmountScores = scoresByMovie['score'].min()              # Minima cantidad de scores: 396
movieIdMinAmountScores = idx_min                            # movieId con minima cantidad de scores: as4940

In [ ]:
# df_v: Peliculas/series ordenadas por mas a menos calificaciones
df_v = df.groupby(['movieId']).count()
df_v.sort_values('userId', ascending = False, inplace = True)
df_v['viewed'] = df_v['userId']
df_v.drop(columns = ['userId','score','platform','title'], inplace = True)
df_v.head()

In [ ]:
# Agrega los title al df_v
titles = titles.set_index('movieId')
df_v['title'] = titles.loc[df_v.index].title
df_v

In [ ]:
movieMaxAmountScores = df_v.loc[movieIdMaxAmountScores, 'title'] 
movieMinAmountScores = df_v.loc[movieIdMinAmountScores, 'title'] 
print('Movie with Maximum amount of scores:',movieMaxAmountScores)
print('Movie with Minimum amount of scores:',movieMinAmountScores)

In [ ]:
# Grafica la cantidad de vistas
plt.figure(figsize = (6,3))
df_v.viewed[df_v.viewed<600].hist(log = True)
plt.title('Distribution for amount of views')
plt.show()

## Conclusiones

In [ ]:
# df.to_csv('data/merge.csv', index=False)
df.to_parquet('data/merge.parquet', index=False)

In [ ]:
df_title.to_csv('data/df_title.csv', index=False)

In [ ]:
print('EDA conclusions:')
print('Scores amount:',scores_amount)
print('Maximum Average Score (by platform)',maxAvgScore)
print('Minimum Average Score (by platform)',minAvgScore)
print('Users amount:',usersAmount)
print('Maximum amount of scores in a movie:', maxAmountScores)
print('Minimum amount of scores in a movie:', minAmountScores)
print('Movie Id with maximum amount of scores:',movieIdMaxAmountScores)
print('Movie Id with minimum amount of scores:',movieIdMinAmountScores)
print('Movie with Maximum amount of scores:',movieMaxAmountScores)
print('Movie with Minimum amount of scores:',movieMinAmountScores)